In [71]:
import pandas as pd
import numpy as np 

from sklearn.tree import DecisionTreeClassifier

from sklearn.metrics import accuracy_score as acc_sc

In [4]:
st_df = pd.read_csv('C:/TIL/1)crawling/st_default.csv')

In [5]:
st_df.head()

,Unnamed: 0,default,student,balance,income
0,1,No,No,729.526495,44361.62507
1,2,No,Yes,817.180407,12106.13470
2,3,No,No,1073.549164,31767.13895
3,4,No,No,529.250605,35704.49394
4,5,No,No,785.655883,38463.49588


In [34]:
st_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  10000 non-null  int64  
 1   default     10000 non-null  int64  
 2   student     10000 non-null  int64  
 3   balance     10000 non-null  float64
 4   income      10000 non-null  float64
dtypes: float64(2), int64(3)
memory usage: 390.8 KB


In [59]:
st_df = st_df.astype({'balance':'int'})
st_df = st_df.astype({'income':'int'})

In [60]:
st_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype
---  ------      --------------  -----
 0   Unnamed: 0  10000 non-null  int64
 1   default     10000 non-null  int64
 2   student     10000 non-null  int64
 3   balance     10000 non-null  int32
 4   income      10000 non-null  int32
dtypes: int32(2), int64(3)
memory usage: 312.6 KB


In [56]:
# 데이터 전처리 

# label Encoder 
from sklearn.preprocessing import LabelEncoder 

encoder = LabelEncoder()
encoder.fit(st_df['default'])
encoder.fit(st_df['student'])
st_df['default'] = encoder.transform(st_df['default'])
st_df['student'] = encoder.transform(st_df['student'])


In [22]:
print(st_df['default'].value_counts())
print(st_df['student'].value_counts())

0    9667
1     333
Name: default, dtype: int64
0    7056
1    2944
Name: student, dtype: int64


In [24]:
st_df[['default', 'student']]

,default,student
0,0,0
1,0,1
2,0,0
3,0,0
4,0,0
...,...,...
9995,0,0
9996,0,0
9997,0,0
9998,0,0


In [61]:
X_data = st_df[['default', 'student', 'balance']]
y_target = st_df['income']

dt_clf = DecisionTreeClassifier()

In [62]:
X_train = X_data.copy()
y_train = y_target.copy()

dt_clf.fit(X_train, y_train)

DecisionTreeClassifier()

In [33]:
st_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  10000 non-null  int64  
 1   default     10000 non-null  int64  
 2   student     10000 non-null  int64  
 3   balance     10000 non-null  float64
 4   income      10000 non-null  float64
dtypes: float64(2), int64(3)
memory usage: 390.8 KB


In [63]:
# 예측을 수행 
y_pred = dt_clf.predict(X_train)

In [64]:
# 평가지표로 평가 
acc_sc(y_train, y_pred)

0.3329

In [67]:
# x와 y를 분할 

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_data, y_target, test_size=0.2, random_state=123)

In [68]:
# 재학습 
dt_clf.fit(X_train, y_train)

DecisionTreeClassifier()

In [69]:
# 예측수행 
y_pred = dt_clf.predict(X_test)

In [72]:
## 평가지표

print(f'예측정확도f: {np.round(acc_sc(y_test, y_pred),3)}')
print('예측정확도: {0:.3f}'.format(acc_sc(y_test,y_pred)))

예측정확도f: 0.0
예측정확도: 0.000


In [109]:
# 필수 라이브러리 

# 판다스 & 넘파이
import pandas as pd
import numpy as np

# train, test set 분리 + 교차검증 
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score

# 알고리즘 - 의사결정나무
from sklearn.tree import DecisionTreeClassifier

# 평가지표 
from sklearn.metrics import accuracy_score

In [110]:
# 데이터 정의 
st_default_df = pd.read_csv('C:/TIL/1)crawling/st_default.csv')
st_default_df.head(3)

,Unnamed: 0,default,student,balance,income
0,1,No,No,729.526495,44361.62507
1,2,No,Yes,817.180407,12106.13470
2,3,No,No,1073.549164,31767.13895


In [111]:
# 인덱스 열 제거(Unnamed)
st_default_df1 = st_default_df.iloc[:,1:]

In [86]:
st_default_df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   default  10000 non-null  object 
 1   student  10000 non-null  object 
 2   balance  10000 non-null  float64
 3   income   10000 non-null  float64
dtypes: float64(2), object(2)
memory usage: 312.6+ KB


In [112]:
# test data, train data 분리하기 전에 문자열 데이터 인코딩 - 1) label vs 2) one-hot 
# y_label의 경우 default를 예측하는 값이므로 이 값은 label
# 나머지 student는 get_dummies()

X_data = st_default_df1.drop('default', axis=1) # 독립변수 default 제외한 나머지 변수 
y_target = st_default_df1['default'] # 종속변수 default 

In [113]:
# 크기 확인 (행 갯수)
print('X_data의 크기:', X_data.shape[0])
print('y_target의 크기:', y_target.shape[0])

X_data의 크기: 10000
y_target의 크기: 10000


In [114]:
# 독립변수 study 인코딩 (one-hot)
X_data_df = pd.get_dummies(X_data)

In [120]:
# 종속변수 default 인코딩 (label)

from sklearn.preprocessing import LabelEncoder

le_encode = LabelEncoder()
y_target_scaled = le_encode.fit_transform(y_target)

# imbalanced data
pd.Series(y_target_scaled).value_counts()

0    9667
1     333
dtype: int64

In [121]:
# 데이터 분할 - train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_data_df, y_target_scaled, stratify=y_target_scaled)

In [122]:
# 의사결정 나무 적용 

dt_clf = DecisionTreeClassifier()
dt_clf.fit(X_train,y_train)

y_pred = dt_clf.predict(X_test)

In [124]:
# 평가지표

print(f'정확도는 :{accuracy_score(y_pred,y_test)}') 

정확도는 :0.9592
